In [16]:
#import funcs
import funcs

In [22]:
#use recipe string and number of search pages as arguements
#return two pandas DataFrames (top recipes info/recipe text(ingredients and instructions))
top_recipes_df, recipe_text_df = funcs.recipe_search(input('Enter recipe search term: '),2)

### Needs debugging removed ###

Enter recipe search term: jalapeno cornbread
-------------------
Starting scrape for page 1
https://www.allrecipes.com/search?q=jalapeno+cornbread
-------------------
5 elements searched of 24
10 elements searched of 24
15 elements searched of 24
20 elements searched of 24
-------------------
Moving to next page
-------------------
14 recipes found on page 1
-------------------
Starting scrape for page 2
https://www.allrecipes.com/search?jalapeno%20cornbread=jalapeno%20cornbread&offset=24&q=jalapeno%20cornbread
-------------------
5 elements searched of 24
10 elements searched of 24
15 elements searched of 24
20 elements searched of 24
-------------------
7 recipes found on page 2
-------------------
Reached the last page...
All recipes scraped and stored...
21 total recipes found
-------------------


In [23]:
#print top_recipes df here
top_recipes_df = top_recipes_df.reset_index(drop = True)
top_recipes_df

,title,rating,url
0,Jalapeno-Bacon Stuffing,5,https://www.allrecipes.com/recipe/277143/jalap...
1,Jalapeno Creamed Corn Casserole,5,https://www.allrecipes.com/recipe/275829/jalap...
2,Pinto Beans With Mexican-Style Seasonings,5,https://www.allrecipes.com/recipe/231037/pinto...


In [24]:
# requests row index from table above as input
recipe_choice = funcs.recipe_select(top_recipes_df)

# takes a string, the recipe_text_df and top_recipes_df as arguements
# prints ingredients and instructions to screen
funcs.get_recipe(recipe_choice,recipe_text_df, top_recipes_df)

Enter the row index for the recipe you wish to see: 1
Url: 
https://www.allrecipes.com/recipe/275829/jalapeno-creamed-corn-casserole/

----------------------
Ingredients:
----------------------
1 (15 ounce) can whole kernel corn, drained
1 (15 ounce) can cream-style corn
1 (8.5 ounce) package corn muffin mix
1 cup sour cream
½ cup unsalted butter, melted
¼ cup diced pickled jalapeno pepper slices
1 pinch salt


Step 1:
----------------------
Preheat the oven to 350 degrees F (175 degrees C). Grease a 9-inch square baking pan.
----------------------

Step 2:
----------------------
Stir together whole kernel corn, cream-style corn, muffin mix, sour cream, melted butter, jalapeño peppers, and salt in a large bowl until combined. Pour batter into the prepared baking pan.
----------------------

Step 3:
----------------------
Bake in the preheated oven until a toothpick inserted into the center comes out clean, 45 to 60 minutes.
----------------------
